In [1]:
import numpy as np
import glob
import os.path
import sys
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import cv2
import matplotlib.pyplot as plt
import imutils
import pandas as pd
import re
import datetime
from sklearn.decomposition import NMF
from sklearn.utils import shuffle
from skimage import draw
from skimage.io import imread
from scipy.sparse import csr_matrix,linalg
from sklearn.model_selection import train_test_split#import spams

import time
import math
from keras.models import *
from keras.layers import *
from keras.preprocessing import image
import keras.backend as K
import random
from sklearn.metrics import f1_score
from skimage.transform import resize
from skimage.filters import threshold_otsu
from tqdm import tqdm_notebook as tqdm 
#from tensorflow.keras.callbacks import TensorBoard
import keras
from tensorboard import version
from keras.models import model_from_json
from keras.layers.merge import concatenate, add
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf
run_opts = tf.RunOptions(report_tensor_allocations_upon_oom = True)
config = tf.ConfigProto()
#config.gpu_options.per_process_gpu_memory_fraction=0.2
config.gpu_options.allow_growth=True ##to use gpu as needed
config.log_device_placement = True  
sess = tf.Session(config=config)
set_session(sess)

from PIL import Image
Image.MAX_IMAGE_PIXELS = None
 

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
MASK_PATH='/home/vahadaneabhi01/datalab/digest/Colonoscopy_tissue_segment_dataset/tissue-train-pos-v1_mask'
IMAGE_PATH='/home/vahadaneabhi01/datalab/digest/Colonoscopy_tissue_segment_dataset/tissue-train-pos-v1_color_normalized'
IMAGE_WRITE_PATH='/home/vahadaneabhi01/datalab/digest/Colonoscopy_tissue_segment_dataset/input'
MASK_WRITE_PATH='/home/vahadaneabhi01/datalab/digest/Colonoscopy_tissue_segment_dataset/mask'
DISCARD_WRITE_PATH='/home/vahadaneabhi01/datalab/digest/Colonoscopy_tissue_segment_dataset/discard'
def extract_patches(MASK_PATH,IMAGE_PATH,IMAGE_WRITE_PATH,MASK_WRITE_PATH,DISCARD_WRITE_PATH,size=512):
    
    if not os.path.exists(IMAGE_WRITE_PATH):
        os.mkdir(IMAGE_WRITE_PATH)
        print('input directory made')
    else:
        print('input directory exists')

    if not os.path.exists(MASK_WRITE_PATH):
        os.mkdir(MASK_WRITE_PATH)
        print('mask directory made')
    else:
        print('mask directory exists')

    if not os.path.exists(DISCARD_WRITE_PATH):
        os.mkdir(DISCARD_WRITE_PATH)
        print('discard directory made')
    else:
        print('discard directory exists')

    image_list=os.listdir(IMAGE_PATH)

    for image_ID in tqdm(image_list):
        # Format of image_ID : D20180056701_2019-05-21 09_50_58-lv1-27237-28645-8099-3942.jpg
        image_name=os.path.join(IMAGE_PATH,image_ID)
        gray_img=cv2.imread(image_name,0)
        bgr_img=cv2.imread(image_name,1)
        mask_ID=image_ID.split('.')[0]+'_mask.'+image_ID.split('.')[-1]
        # Format of mask_ID: D20180056701_2019-05-21 09_50_58-lv1-27237-28645-8099-3942_mask.jpg
        mask_name=os.path.join(MASK_PATH,mask_ID)
        mask_img=cv2.imread(mask_name,0)
        ret,mask_img=cv2.threshold(mask_img,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)



        th,img_otsu=cv2.threshold(gray_img,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
        temp_x,temp_y=np.where(img_otsu==255)

        for count_x,x in enumerate(range(img_otsu.shape[0]//512)):
            for count_y,y in enumerate(range(img_otsu.shape[1]//512)):

                temp_otsu=img_otsu[count_x*512:(count_x+1)*512,count_y*512:(count_y+1)*512]
                temp_mask=mask_img[count_x*512:(count_x+1)*512,count_y*512:(count_y+1)*512]


                if len(np.where(temp_otsu==0)[0])<0.6*len(np.where(temp_otsu==255)[0]) and len(np.where(temp_mask==255)[0])>=5000:
                    img_write=bgr_img[count_x*512:(count_x+1)*512,count_y*512:(count_y+1)*512,:]

                    write_path_image=os.path.join(IMAGE_WRITE_PATH,image_ID.split('.')[0]+\
                                              "_{}_{}.jpg".format(count_x,count_y))
            # Format of image_ID now: D20180056701_2019-05-21 09_50_58-lv1-27237-28645-8099-3942_<count_x>_<count_y>.jpg
                    cv2.imwrite(write_path_image,img_write)
                    write_path_mask=os.path.join(MASK_WRITE_PATH,mask_ID.split('_mask')[0]+\
                                                  "_{}_{}".format(count_x,count_y)+'_mask.jpg')
            # Format of mask_ID now: D20180056701_2019-05-21 09_50_58-lv1-27237-28645-8099-3942_<count_x>_<count_y>_mask.jpg
                    cv2.imwrite(write_path_mask,temp_mask)
                else:
                    write_path_discard=os.path.join(DISCARD_WRITE_PATH,mask_ID.split('.')[0]+\
                                                  "{}_{}.jpg".format(count_x,count_y))
                    cv2.imwrite(write_path_discard,temp_mask)

#extract_patches(MASK_PATH,IMAGE_PATH,IMAGE_WRITE_PATH,MASK_WRITE_PATH,DISCARD_WRITE_PATH)

MASK_PATH_NN='/home/vahadaneabhi01/datalab/digest/Colonoscopy_tissue_segment_dataset/tissue-train-pos-v1_mask'
IMAGE_PATH_NN='/home/vahadaneabhi01/datalab/digest/Colonoscopy_tissue_segment_dataset/tissue-train-pos-v1'
IMAGE_WRITE_PATH_NN='/home/vahadaneabhi01/datalab/digest/Colonoscopy_tissue_segment_dataset/input_non_normalized'
MASK_WRITE_PATH_NN='/home/vahadaneabhi01/datalab/digest/Colonoscopy_tissue_segment_dataset/mask_non_normalized'
DISCARD_WRITE_PATH_NN='/home/vahadaneabhi01/datalab/digest/Colonoscopy_tissue_segment_dataset/discard_non_normalized'

#extract_patches(MASK_PATH_NN,IMAGE_PATH_NN,IMAGE_WRITE_PATH_NN,MASK_WRITE_PATH_NN,DISCARD_WRITE_PATH_NN)

In [7]:
MASK_PATH='/home/vahadaneabhi01/datalab/digest/Colonoscopy_tissue_segment_dataset/tissue-train-pos-v1_mask'
IMAGE_PATH='/home/vahadaneabhi01/datalab/digest/Colonoscopy_tissue_segment_dataset/tissue-train-pos-v1_color_normalized'
IMAGE_OVERLAP_WRITE_PATH='/home/vahadaneabhi01/datalab/digest/Colonoscopy_tissue_segment_dataset/input_overlap'
MASK_OVERLAP_WRITE_PATH='/home/vahadaneabhi01/datalab/digest/Colonoscopy_tissue_segment_dataset/mask_overlap'
DISCARD_OVERLAP_WRITE_PATH='/home/vahadaneabhi01/datalab/digest/Colonoscopy_tissue_segment_dataset/discard_overlap'
def extract_patches(MASK_PATH,IMAGE_PATH,IMAGE_WRITE_PATH,MASK_WRITE_PATH,DISCARD_WRITE_PATH,size=512):
    
    if not os.path.exists(IMAGE_WRITE_PATH):
        os.mkdir(IMAGE_WRITE_PATH)
        print('input directory made')
    else:
        print('input directory exists')

    if not os.path.exists(MASK_WRITE_PATH):
        os.mkdir(MASK_WRITE_PATH)
        print('mask directory made')
    else:
        print('mask directory exists')

    if not os.path.exists(DISCARD_WRITE_PATH):
        os.mkdir(DISCARD_WRITE_PATH)
        print('discard directory made')
    else:
        print('discard directory exists')

    image_list=os.listdir(IMAGE_PATH)

    for image_ID in tqdm(image_list):
        # Format of image_ID : D20180056701_2019-05-21 09_50_58-lv1-27237-28645-8099-3942.jpg
        image_name=os.path.join(IMAGE_PATH,image_ID)
        gray_img=cv2.imread(image_name,0)
        bgr_img=cv2.imread(image_name,1)
        mask_ID=image_ID.split('.')[0]+'_mask.'+image_ID.split('.')[-1]
        # Format of mask_ID: D20180056701_2019-05-21 09_50_58-lv1-27237-28645-8099-3942_mask.jpg
        mask_name=os.path.join(MASK_PATH,mask_ID)
        mask_img=cv2.imread(mask_name,0)
        ret,mask_img=cv2.threshold(mask_img,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)



        th,img_otsu=cv2.threshold(gray_img,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
        temp_x,temp_y=np.where(img_otsu==255)
        range_x=int((img_otsu.shape[0]-512)/256) +1
        range_y=int((img_otsu.shape[1]-512)/256) +1

        for count_x,x in enumerate(range(range_x)):
            x_start=count_x*256
            x_end=x_start+512
            for count_y,y in enumerate(range(range_y)):
                y_start=count_y*256
                y_end=y_start+512
                
                                   

                temp_otsu=img_otsu[x_start:x_end,y_start:y_end]
                temp_mask=mask_img[x_start:x_end,y_start:y_end]


                if len(np.where(temp_otsu==0)[0])<0.6*len(np.where(temp_otsu==255)[0]) and len(np.where(temp_mask==255)[0])>=5000:
                    img_write=bgr_img[x_start:x_end,y_start:y_end,:]

                    write_path_image=os.path.join(IMAGE_WRITE_PATH,image_ID.split('.')[0]+\
                                              "_{}_{}.jpg".format(count_x,count_y))
            # Format of image_ID now: D20180056701_2019-05-21 09_50_58-lv1-27237-28645-8099-3942_<count_x>_<count_y>.jpg
                    cv2.imwrite(write_path_image,img_write)
                    write_path_mask=os.path.join(MASK_WRITE_PATH,mask_ID.split('_mask')[0]+\
                                                  "_{}_{}".format(count_x,count_y)+'_mask.jpg')
            # Format of mask_ID now: D20180056701_2019-05-21 09_50_58-lv1-27237-28645-8099-3942_<count_x>_<count_y>_mask.jpg
                    cv2.imwrite(write_path_mask,temp_mask)
                else:
                    write_path_discard=os.path.join(DISCARD_WRITE_PATH,mask_ID.split('.')[0]+\
                                                  "{}_{}.jpg".format(count_x,count_y))
                    cv2.imwrite (write_path_discard,temp_mask)

extract_patches(MASK_PATH,IMAGE_PATH,IMAGE_OVERLAP_WRITE_PATH,MASK_OVERLAP_WRITE_PATH,DISCARD_OVERLAP_WRITE_PATH,size=512)



input directory made
mask directory made
discard directory made


In [3]:
class DataGenerator(keras.utils.Sequence):

    def __init__(self,image_IDs,IMAGE_PATH,MASK_PATH,batch_size=1,sample_wise=False,feature_wise=False,scale=1,\
                 shuffle=True):
        print("Number of sample : {}".format(len(image_IDs)))

        self.image_IDs = image_IDs
        self.IMAGE_PATH=IMAGE_PATH
        self.MASK_PATH=MASK_PATH
        self.batch_size=batch_size
        self.sample_wise=sample_wise
        self.scale=scale
        self.feature_wise=feature_wise
        if self.feature_wise:
            self.mean_value=np.array([0.788582,0.69378152,0.81263777])
            self.std_value=np.array([0.20195358,0.28290182,0.16003539])
            
           
        
    
        
        self.shuffle = shuffle
        self.on_epoch_end()
    

    def __len__(self):
        return int(np.ceil(len(self.image_IDs) / self.batch_size))
    

    def __getitem__(self, index):
    
        image_ID_temp= self.indices[index]
    
        X, y = self.__data_generation(image_ID_temp)
            

        return X, y

    def on_epoch_end(self):
        
        self.indices = np.arange(len(self.image_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indices)

    def __data_generation(self, image_ID_temp):
        
        img_path=os.path.join(self.IMAGE_PATH,self.image_IDs[image_ID_temp])
        
        mask_name=self.image_IDs[image_ID_temp].split('.')[0]+'_mask.jpg'
        mask_path=os.path.join(self.MASK_PATH,mask_name)
        
        X = imread(img_path)
        X=resize(X,(1024,1024,3))
        
        if self.scale!=1 and self.sample_wise==True:
            sys.exit("Scale and sample_wise, both cannot be implemneted")
        
        X=X*self.scale
        
        if self.feature_wise==True:
            X=np.array(X,np.float64)
            X[:,:,0]=(X[:,:,0]-self.mean_value[0])/self.std_value[0]
            X[:,:,1]=(X[:,:,1]-self.mean_value[1])/self.std_value[2]
            X[:,:,2]=(X[:,:,2]-self.mean_value[2])/self.std_value[2]
        if self.sample_wise==True:
            X=np.array(X,np.float64)
            X[:,:,0]=(X[:,:,0]-np.mean(X[:,:,0]))/np.std(X[:,:,0])
            X[:,:,1]=(X[:,:,1]-np.mean(X[:,:,1]))/np.std(X[:,:,1])
            X[:,:,2]=(X[:,:,2]-np.mean(X[:,:,2]))/np.std(X[:,:,2])
        X=np.expand_dims(X,axis=0)
        y=imread(mask_path,as_gray=True)
        thresh=threshold_otsu(y)
        y=y>thresh
        y=resize(y,(1024,1024))
        y=y.astype(np.float_)

        y=np.expand_dims(y,axis=0)
        y=np.expand_dims(y,axis=3)


        return X,y

In [4]:
INPUT_IMAGE_PATH='/home/vahadaneabhi01/datalab/digest/Colonoscopy_tissue_segment_dataset/tissue-train-pos-v1'
OUTPUT_MASK_PATH='/home/vahadaneabhi01/datalab/digest/Colonoscopy_tissue_segment_dataset/tissue-train-pos-v1_mask'

image_list=os.listdir(INPUT_IMAGE_PATH)
temp=image_list.copy()
train_list=image_list[:int(len(image_list)*0.8)]
test_list=image_list[int(len(image_list)*0.8):]
train_generator=DataGenerator(train_list,INPUT_IMAGE_PATH,OUTPUT_MASK_PATH,batch_size=1,feature_wise=True,scale=1)
test_generator=DataGenerator(test_list,INPUT_IMAGE_PATH,OUTPUT_MASK_PATH,batch_size=1,feature_wise=True,scale=1)

Number of sample : 200
Number of sample : 50


In [5]:
!nvidia-smi


Tue Aug 20 18:43:42 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.39       Driver Version: 418.39       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:3D:00.0 Off |                    0 |
| N/A   36C    P0    69W / 300W |    627MiB / 32480MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [12]:
class CosineWithRestart(keras.callbacks.Callback):
    
    def __init__(self,
                 min_lr,
                 max_lr,
                 steps_per_epoch,
                 lr_decay=1,
                 cycle_length=100,
                 mult_factor=2):

        self.min_lr = min_lr
        self.max_lr = max_lr
        self.lr_decay = lr_decay

        self.batch_since_restart = 0
        self.next_restart = cycle_length

        self.steps_per_epoch = steps_per_epoch

        self.cycle_length = cycle_length
        self.mult_factor = mult_factor

        self.history = {}

    def clr(self):
        
        fraction_to_restart = self.batch_since_restart / (self.steps_per_epoch * self.cycle_length)
        lr = self.min_lr + 0.5 * (self.max_lr - self.min_lr) * (1 + np.cos(fraction_to_restart * np.pi))
        return lr

    def on_train_begin(self, logs={}):
        logs = logs or {}
        K.set_value(self.model.optimizer.lr, self.max_lr)

    def on_batch_end(self, batch, logs={}):
        '''Record previous batch statistics and update the learning rate.'''
        logs = logs or {}
        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)

        self.batch_since_restart += 1
        
        K.set_value(self.model.optimizer.lr, self.clr())

    def on_epoch_end(self, epoch, logs={}):
        '''Check for end of current cycle, apply restarts when necessary.'''
        if epoch + 1 == self.next_restart:
            self.batch_since_restart = 0
            self.cycle_length = np.ceil(self.cycle_length * self.mult_factor)
                
            self.next_restart += self.cycle_length
            self.max_lr *= self.lr_decay
            self.best_weights = self.model.get_weights()

    def on_train_end(self, logs={}):
        '''Set weights to the values from the end of the most recent cycle for best performance.'''
        self.model.set_weights(self.best_weights)
        
        
class CheckLR(tf.keras.callbacks.Callback):
    def on_epoch_begin(self,epoch,logs):
        print("LR : {}".format(K.eval(self.model.optimizer.lr)))
        

def dice_coef(y_true, y_pred):
    
    y_pred = K.round(y_pred)
    intersection = K.sum(y_true * y_pred, axis=[1,2,3])
    union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3])
    return K.mean( (2. * intersection +K.epsilon()) / (union + K.epsilon()), axis=0)

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

def f1_score(y_true,y_pred):
    y_pred=K.squeeze(y_pred[0],axis=2)
    y_true=K.squeeze(y_true[0],axis=2)
    
    y_pred_rounded = K.round(y_pred)

    tp = K.sum(y_true * y_pred_rounded)
    fp = K.sum((1-y_true)*y_pred_rounded)
    fn = K.sum(y_true*(1-y_pred_rounded))

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    return K.mean(f1)



In [13]:
def unet(pretrained_weights = None,input_size = (None,None,3)):
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = Model(input = inputs, output = conv10)
    
    OPT=keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0000, amsgrad=False)

    model.compile(optimizer = OPT, loss = 'binary_crossentropy', metrics = ['accuracy',dice_coef,f1_score])
    

    return model



In [14]:
model=unet()
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_49 (Conv2D)              (None, None, None, 6 1792        input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_50 (Conv2D)              (None, None, None, 6 36928       conv2d_49[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_9 (MaxPooling2D)  (None, None, None, 6 0           conv2d_50[0][0]                  
__________________________________________________________________________________________________
conv2d_51 

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:43: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`


In [15]:
TRAIN_STEPS=train_generator.__len__()
TEST_STEPS=test_generator.__len__()

model_start_date=datetime.datetime.now().strftime("%Y_%m_%d")
dir_name = os.path.join(os.getcwd(),"model_resize_{}".format(model_start_date))

if os.path.exists(dir_name):
    print("model already exists in {}".format("model_resize_{}".format(model_start_date)))
else:
    os.mkdir(dir_name)
    print(" dir {} made".format("model_resize_{}".format(model_start_date)))




model already exists in model_resize_2019_08_20


In [16]:
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1,mode='min',min_delta=0.01,\
                     patience=4,cooldown=0,min_lr= 1e-12,verbose=1)
def step_decay(epoch):
    initial_lrate = 0.01
    drop = 0.5
    epochs_drop = 2
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate

lr_sceduler=keras.callbacks.LearningRateScheduler(step_decay, verbose=1)

mc_cb=keras.callbacks.ModelCheckpoint('{}/model_optimum_{}.hdf5'.\
                                      format(dir_name,datetime.datetime.now().strftime("%Y_%m_%d")),\
                                      monitor='val_acc', verbose=1, save_best_only=True, \
                                      save_weights_only=True, mode='max', period=1)


schedule=CosineWithRestart(1e-06,0.01,TRAIN_STEPS,lr_decay=0.0004,cycle_length=2,mult_factor=2)
lr=CheckLR()
callbacks=[mc_cb,lr,lr_sceduler]

In [17]:

history=model.fit_generator(generator=train_generator,epochs=10,verbose=1,\
                            callbacks=callbacks,\
                                 validation_data=test_generator,use_multiprocessing=True)
#model.fit(train_x, train_y, epochs=600, batch_size=1, verbose=1, shuffle=False,validation_data=(test_x,test_y))

fig=plt.figure()
plt.plot(history.history['loss'],label='training_loss')
plt.plot(history.history['val_loss'],label='validation_loss')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('loss')
fig.savefig('{}/plot_loss_{}.PNG'.format(dir_name,model_start_date))
f1=history.history['f1_score']
f1_test=history.history['val_f1_score']
fig2=plt.figure()
plt.plot(f1,label='training')
plt.plot(f1_test,label='testing')
plt.legend()
print(f1[-1],f1_test[-1])
fig2.savefig('{}/plot_f1_macro_{}.PNG'.format(dir_name,model_start_date))
model_json = model.to_json()
with open("{}/model_{}.json".format(dir_name,model_start_date), "w") as json_file:
    json_file.write(model_json)
model.save_weights("{}/model_final_weights_{}.h5".format(dir_name,model_start_date))
print("Saved model to disk")


Epoch 1/10
LR : 0.009999999776482582

Epoch 00001: LearningRateScheduler setting learning rate to 0.01.
200/200 [==============================] - 2829s 14s/step - loss: 1.1493 - acc: 0.9271 - dice_coef: 0.4303 - f1_score: 3.0677e-04 - val_loss: 0.9293 - val_acc: 0.9417 - val_dice_coef: 0.5000 - val_f1_score: 0.0000e+00

Epoch 00001: val_acc improved from -inf to 0.94174, saving model to /datalab/digest/Colonoscopy_tissue_segment_dataset/model_resize_2019_08_20/model_optimum_2019_08_20.hdf5
Epoch 2/10
LR : 0.009999999776482582

Epoch 00002: LearningRateScheduler setting learning rate to 0.005.
200/200 [==============================] - 2590s 13s/step - loss: 1.1631 - acc: 0.9272 - dice_coef: 0.4300 - f1_score: 0.0000e+00 - val_loss: 0.9293 - val_acc: 0.9417 - val_dice_coef: 0.5000 - val_f1_score: 0.0000e+00

Epoch 00002: val_acc did not improve from 0.94174
Epoch 3/10
LR : 0.004999999888241291

Epoch 00003: LearningRateScheduler setting learning rate to 0.005.
199/200 [================

KeyboardInterrupt: 

In [ ]:
x,y=train_generator.__getitem__(3)
plt.imshow(x[0])

In [ ]:
def get_mean_std():
    INPUT_IMAGE_PATH='/home/vahadaneabhi01/datalab/digest/Colonoscopy_tissue_segment_dataset/tissue-train-pos-v1'
    OUTPUT_MASK_PATH='/home/vahadaneabhi01/datalab/digest/Colonoscopy_tissue_segment_dataset/tissue-train-pos-v1_mask'
    image_list=os.listdir(INPUT_IMAGE_PATH)
    x=[]
    for i in tqdm(image_list):
        path=os.path.join(INPUT_IMAGE_PATH,i)
        img=imread(path)
        img=resize(img,(1024,1024,3))
        x.append(img)

    x=np.array(x)
    mean=np.mean(x,axis=(1,0,2))
    std=np.std(x,axis=(0,1,2))
    print(mean,std)

    